In [46]:
import networkx as nx
import numpy as np
import Visualizer

In [47]:
G = nx.gnm_random_graph(1280, 2340, seed=42)

In [48]:
char_path_length = 'not connected'
pos_max = np.argmax([len(listElem) for listElem in sorted(nx.connected_components(G))])
char_path_length_component = [nx.average_shortest_path_length(G.subgraph(g)) for g in sorted(nx.connected_components(G))][pos_max]

density = nx.density(G)

transitivity = nx.transitivity(G)
clustering_coef = nx.average_clustering(G)

n_connected_comp = nx.number_connected_components(G)
size_connected_comp = sum([len(listElem) for listElem in sorted(nx.connected_components(G))])/len(sorted(nx.connected_components(G)))

print(f'original characteristic path length {char_path_length}, \noriginal characteristic path length of biggest component {char_path_length_component},\noriginal network density {density},\noriginal transitivity {transitivity}, \noriginal average clustering coefficient {clustering_coef}, \noriginal number of connected components {n_connected_comp}, \noriginal average size of a connected component {size_connected_comp}')

original characteristic path length not connected, 
original characteristic path length of biggest component 5.638166659337305,
original network density 0.0028586786551993747,
original transitivity 0.0021337126600284497, 
original average clustering coefficient 0.0026109871031746034, 
original number of connected components 35, 
original average size of a connected component 36.57142857142857


In [49]:
def simulate_sirs(G,beta,gamma,xi,f,lst_initial_immune,lst_initial_infected):
    attDict = {}
    for node in lst_initial_immune:
        attDict[node] = [0]
    for node in lst_initial_infected:
        attDict[node] = [1]
    t = 0
    D = []
    D.append([len(lst_initial_immune), len(lst_initial_infected), 0])
    while t < f:
        S = 0
        I = 0
        R = 0
        for node in list(attDict.keys()):
            if attDict[node][t] == 0:
                if np.random.random() < beta:
                    attDict[node].append(1)
                    I += 1
                else:
                    attDict[node].append(0)
                    S += 1
            if attDict[node][t] == 1:
                if np.random.random() < gamma:
                    attDict[node].append(2)
                    R += 1
                else:
                    attDict[node].append(1)
                    I += 1
            if attDict[node][t] == 2:
                if np.random.random() < xi:
                    attDict[node].append(0)
                    S += 1
                else:
                    attDict[node].append(2)
                    R += 1
        D.append([S,I,R])
        t+=1
    nx.set_node_attributes(G,attDict,"state")
    return D

In [50]:
D = simulate_sirs(G, 0.3, 0.4, 0.1, 10, list(G.nodes)[:640], list(G.nodes)[640:])

In [51]:
Visualizer.showSIRS(G, "test_simulation.html", 0.3, 0.4, 0.1, 10, D)

Nodes positions computed.
Edges trace computed.
Nodes trace computed.
Frames computed.
Figure generated.
Visualization rendering done
File saved.


In [52]:
Visualizer.showData(D, "test_data.html", 0.3, 0.4, 0.1)